In [0]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

In [15]:
from google.colab import drive
drive.mount("ORIE4741Project", force_remount=True)

Mounted at ORIE4741Project


# Prediction method

We will first combine the dataset, the features are the zipcode, longitude, latitude, indoor complaint change, cafe change, inspection change. 
We want to predict two types of lables, 
The first kind of labels are the catogorized data for the change of housing prices in certain years in a zipcode, catogirized intor 10 bins, <-80%, -80% to -60%, etc. to 60% to 80%, and >100% 
The second kind of labels are the direct percentage of change of housing prices in certain years in a zipcode.

The regression method we will be using are random forest for the categorical change labels, random forest regressor for the percentage labels.

For the data in 2016-2017 change, we will also take 2015-2016 change into the input features.
For the data in 2017-2018 change, we will also take 2015-2016 change and 2015-2016 change into the input features.

We will then use our developed and trained model to validate itself by predicting the housing prices change in 2017-2018 in the validation dataset.  

In [0]:
# x predict y, only considering data from 2015-2016
# x : zipcode(int), longitude(double), latitude(double), indoor complaint change(double), cafe change(double), inspection change(double)
# y : housing price change(double)
# try to predict on 2016-2017, and see the error rate

# x predict y, only considering data from 2016-2017
# x : zipcode(int), longitude(double), latitude(double), indoor complaint change(double), cafe change(double), inspection change(double)
# y : housing price change(double)2016-2017
# try to predict on 2017-2018, and see the error rate
# try use another x with 2015-2016 data
# x : zipcode(int), longitude(double), latitude(double), indoor complaint change(double)[15-16, 16-17], cafe change(double)[15-16, 16-17], inspection change(double)[15-16, 16-17]
# y : housing price change(double)2016-2017
# try to predict on 2017-2018, and see the error rate

# Import all the data
# Indoor complaint

In [17]:
indoor_complaint = pd.read_csv("/content/ORIE4741Project/My Drive/ORIE4741Project/Data/New Data/DOHMH_Indoor_Environmental_Complaints.csv", encoding='latin-1')

indoor_complaint['Year_After_2015'] = indoor_complaint.Date_Received.str.contains('2015') | indoor_complaint.Date_Received.str.contains('2016') | indoor_complaint.Date_Received.str.contains('2017')| indoor_complaint.Date_Received.str.contains('2018')
indoor_complaint = indoor_complaint[indoor_complaint['Year_After_2015']]

indoor_complaint = indoor_complaint[indoor_complaint.Latitude.notnull()]
indoor_complaint = indoor_complaint[indoor_complaint.Longitude.notnull()]
indoor_complaint = indoor_complaint.drop(['Year_After_2015'], axis = 1)

years = indoor_complaint.Date_Received
indoor_complaint['year'] = years.apply(lambda x: x[6:10])

indoor_complaint_by_zipcode = indoor_complaint.groupby(['Incident_Address_Zip', 'year']).count()[['Complaint_Number']]
indoor_complaint_by_zipcode = indoor_complaint_by_zipcode.reset_index()

zipcodes = indoor_complaint_by_zipcode.groupby('Incident_Address_Zip')['Incident_Address_Zip'].unique()
zipcodes_year_count = indoor_complaint_by_zipcode.groupby('Incident_Address_Zip')['Incident_Address_Zip'].count()
indoor_complaint_change = pd.DataFrame(columns=['Zipcode', '2016_change', '2017_change','2018_change'])
indoor_complaint_by_zipcode.loc[(indoor_complaint_by_zipcode['Incident_Address_Zip'] == '10000') & (indoor_complaint_by_zipcode['year'] == '2016')]['Complaint_Number'][0]

for z in zipcodes:
  # check if zipcode valid
  if len(z[0]) == 5:
    # if all four years data exist
    if zipcodes_year_count[z[0]] == 4:
      # complaint_2015 = indoor_complaint_by_zipcode.at()
      complaint_2015 = indoor_complaint_by_zipcode.loc[(indoor_complaint_by_zipcode['Incident_Address_Zip'] == z[0]) & (indoor_complaint_by_zipcode['year'] == '2015')]['Complaint_Number']
      complaint_2016 = indoor_complaint_by_zipcode.loc[(indoor_complaint_by_zipcode['Incident_Address_Zip'] == z[0]) & (indoor_complaint_by_zipcode['year'] == '2016')]['Complaint_Number']
      complaint_2017 = indoor_complaint_by_zipcode.loc[(indoor_complaint_by_zipcode['Incident_Address_Zip'] == z[0]) & (indoor_complaint_by_zipcode['year'] == '2017')]['Complaint_Number']
      complaint_2018 = indoor_complaint_by_zipcode.loc[(indoor_complaint_by_zipcode['Incident_Address_Zip'] == z[0]) & (indoor_complaint_by_zipcode['year'] == '2018')]['Complaint_Number']
      change_2016 = complaint_2016.iloc[0] / complaint_2015.iloc[0] - 1
      change_2017 = complaint_2017.iloc[0] / complaint_2016.iloc[0] - 1
      change_2018 = complaint_2018.iloc[0] / complaint_2017.iloc[0] - 1
    # one of the four years data exist
    else:
      change_2016 = 0
      change_2017 = 0
      change_2018 = 0
    indoor_complaint_change = indoor_complaint_change.append({'Zipcode': z[0], '2016_change': change_2016, '2017_change': change_2017, '2018_change': change_2018}, ignore_index=True)

print(indoor_complaint_change)

    Zipcode 2016_change 2017_change 2018_change
0     10000           0           0           0
1     10001   -0.333333  -0.0454545    0.833333
2     10002    0.254717   -0.293233   -0.287234
3     10003   0.0898876   -0.154639    0.304878
4     10004       -0.25    0.666667         2.2
..      ...         ...         ...         ...
205   11691    0.333333    0.428571        -0.1
206   11692   -0.153846           0  -0.0909091
207   11693   -0.166667         1.4     1.41667
208   11694   0.0666667     -0.0625     1.26667
209   11697           0           0           0

[210 rows x 4 columns]


# Import cafe data

In [18]:
cafe = pd.read_csv("/content/ORIE4741Project/My Drive/ORIE4741Project/Data/New Data/Sidewalk_Caf__Licenses_and_Applications.csv")
cafe_clean = cafe[cafe["LIC_STATUS"] == "Active"][["ZIP", "SUBMIT_DATE"]]
cafe_clean["year"] = pd.DatetimeIndex(cafe_clean['SUBMIT_DATE']).year
cafe_clean = cafe_clean.groupby(['ZIP', 'year']).count()[['SUBMIT_DATE']]
cafe_clean = cafe_clean.reset_index()
zipcodes = cafe_clean.groupby('ZIP')['ZIP'].unique()
zipcodes_year_count = cafe_clean.groupby('ZIP')['ZIP'].count()
cafe_change = pd.DataFrame(columns=['Zipcode', '2016_change', '2017_change','2018_change'])
# cafe_clean.loc[(cafe_clean['ZIP'] == '10000') & (cafe_clean['year'] == 2016)]['SUBMIT_DATE'][0]
for z in zipcodes:
  # check if zipcode valid
  if z[0] >= 10000:
    # if all four years data exist
    if zipcodes_year_count[z[0]] >= 4:
      # complaint_2015 = indoor_complaint_by_zipcode.at()
      try:
        cafe_2015 = cafe_clean.loc[(cafe_clean['ZIP'] == z[0]) & (cafe_clean['year'] == 2015)]['SUBMIT_DATE']
      except: 
        cafe_2015 = 0
      try: 
        cafe_2016 = cafe_clean.loc[(cafe_clean['ZIP'] == z[0]) & (cafe_clean['year'] == 2016)]['SUBMIT_DATE']
      except:
        cafe_2016 = 0
      try:
        cafe_2017 = cafe_clean.loc[(cafe_clean['ZIP'] == z[0]) & (cafe_clean['year'] == 2017)]['SUBMIT_DATE']
      except:
        cafe_2017 =0
      try:
        cafe_2018 = cafe_clean.loc[(cafe_clean['ZIP'] == z[0]) & (cafe_clean['year'] == 2018)]['SUBMIT_DATE']
      except:
        cafe_2018 = 0
      try:
        change_2016 = (cafe_2016.iloc[0]-cafe_2015.iloc[0]) / cafe_2015.iloc[0]
      except: 
        change_2016 = 0
      try: 
        change_2017 = (cafe_2017.iloc[0]-cafe_2016.iloc[0]) / cafe_2016.iloc[0]
      except:
        change_2017 = 0
      try:
        change_2018 = (cafe_2018.iloc[0]-cafe_2017.iloc[0]) / cafe_2017.iloc[0]
      except:
        change_2018 = 0

      
    # one of the four years data exist
    else:
      change_2016 = 0
      change_2017 = 0
      change_2018 = 0
    cafe_change = cafe_change.append({'Zipcode': z[0], '2016_change': change_2016, '2017_change': change_2017, '2018_change': change_2018}, ignore_index=True)

print(cafe_change)

   Zipcode 2016_change 2017_change 2018_change
0    10001           0           0           0
1    10002           0           0           0
2    10003           0           0         5.2
3    10004           0           0           0
4    10006           0           0           0
..     ...         ...         ...         ...
87   11375           0           0           0
88   11377           0           0           0
89   11379           0           0           0
90   11421           0           0           0
91   11432           0           0           0

[92 rows x 4 columns]


# Import housing price data

In [19]:
rent = pd.read_excel("/content/ORIE4741Project/My Drive/ORIE4741Project/Data/New Data/Zillow Rent Prices.xls")
rent_clean = rent.drop([0, 1, 2])
rent_clean = rent_clean.rename(columns={"New York, NY - All Homes": "zipcode", "Unnamed: 48": "year over year"})
rent_by_zipcode = rent_clean[["zipcode", "year over year"]]
print(rent_by_zipcode)

    zipcode year over year
3     10001        0.12936
4     10002       0.112807
5     10003     -0.0939048
6     10004       0.125864
7     10005      0.0695574
..      ...            ...
184   11691              0
185   11692    -0.00656234
186   11693     -0.0114943
187   11694    -0.00701139
188   11697     -0.0285855

[186 rows x 2 columns]


In [20]:
rent_by_zipcode.describe()

,zipcode,year over year
count,186,186
unique,186,185
top,11103,---
freq,1,2


# Regression Method
## generate the data

In [21]:
!pip install uszipcode

In [0]:
from sklearn.model_selection import train_test_split
from uszipcode import Zipcode, SearchEngine

bins = [-np.inf, -0.08, -0.06, -0.04, -0.02, 0, 0.02, 0.04, 0.06, 0.08, np.inf]
ranges = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# create a new column of median range
rent_by_zipcode = rent_by_zipcode.replace('---', '0')
rent_by_zipcode['year over year'] = rent_by_zipcode['year over year'].astype(float)
rent_by_zipcode['PriceChange'] = pd.cut(rent_by_zipcode['year over year'], bins, labels=ranges)
rent_by_zipcode = rent_by_zipcode.dropna()

indoor_complaint_change_renamed = indoor_complaint_change.rename(columns={"2016_change": "icc2016_change", "2017_change": "icc2017_change","2018_change": "icc2018_change"})
cafe_change_renamed = cafe_change.rename(columns={"2016_change": "cc2016_change", "2017_change": "cc2017_change","2018_change": "cc2018_change"})

# general_table = pd.merge(indoor_complaint_change_renamed, cafe_change_renamed, left_on='Zipcode', right_on='Zipcode', how='left')
general_table = pd.merge(indoor_complaint_change_renamed, rent_by_zipcode, left_on='Zipcode', right_on='zipcode', how='left')
general_table = general_table.dropna()

# add longitude and latitude data to data table
search = SearchEngine(simple_zipcode=True)

zipcode_data = search.by_zipcode("10001").to_dict()

lng_list = []
lat_list = []
for index, row in general_table.iterrows():
  zipcode_data = search.by_zipcode(row["Zipcode"]).to_dict()
  lng = zipcode_data['lng']
  lat = zipcode_data['lat']
  lng_list.append(lng)
  lat_list.append(lat)

general_table.insert(1, "longtitude", lng_list) 
general_table.insert(2, "latitude", lat_list) 
general_table['Zipcode'] = general_table['Zipcode'].astype(int)

general_table = general_table.dropna()

In [23]:
kaggle_income = pd.read_csv("/content/ORIE4741Project/My Drive/ORIE4741Project/Data/kaggle_income.csv", encoding='latin-1')
nyc_income = kaggle_income[kaggle_income['City'] == 'New York']
nyc_income = nyc_income[nyc_income['State_Name'] == "New York"]
restaurant_inspection = pd.read_csv("/content/ORIE4741Project/My Drive/ORIE4741Project/Data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv", encoding='latin-1')
restaurant_inspection['ZIPCODE'] = restaurant_inspection['ZIPCODE'].fillna(0.0).astype(int)
restaurant_inspection['SCORE'] =  restaurant_inspection['SCORE'].dropna()
inspection_by_zipcode = restaurant_inspection.groupby(["ZIPCODE"]).mean()[['SCORE']]
income_inspection_by_zipcode = pd.merge(left=inspection_by_zipcode,right=nyc_income, left_on='ZIPCODE', right_on='Zip_Code')

nyc_restaurant_week = pd.read_csv("/content/ORIE4741Project/My Drive/ORIE4741Project/Data/restaurant_week_2018.csv", encoding='latin-1')
nyc_restaurant_week = nyc_restaurant_week.drop(['google_map','website','description','phone'], axis=1)
nyc_restaurant_week = nyc_restaurant_week.dropna()
nyc_restaurant_week["postal_code"] = nyc_restaurant_week["postal_code"].dropna().astype(int)
nyc_restaurant_week['average_review'] =  nyc_restaurant_week['average_review'].dropna()
rating_by_zipcode = nyc_restaurant_week.groupby(["postal_code"]).mean()[['average_review']]

income_inspection_rating_by_zipcode = pd.merge(left=income_inspection_by_zipcode,right=rating_by_zipcode, left_on='Zip_Code', right_on='postal_code')

bins = [0, 50000, 100000, 150000, 200000, 250000, 300000, np.inf]
ranges = [1, 2, 3, 4, 5, 6, 7]
income_inspection_rating_by_zipcode['IncomeMedianRange'] = pd.cut(income_inspection_rating_by_zipcode['Median'], bins, labels=ranges)
income_inspection_rating_by_zipcode = income_inspection_rating_by_zipcode.dropna()
income_inspection_rating_by_zipcode = income_inspection_rating_by_zipcode[['Zip_Code', 'Median', 'IncomeMedianRange', 'average_review', 'SCORE']]

print(income_inspection_rating_by_zipcode.head())

   Zip_Code  Median IncomeMedianRange  average_review      SCORE
0     10001   40262                 1        4.501787  18.908554
1     10002   91469                 2        4.605711  19.714524
2     10002  300000                 6        4.605711  19.714524
3     10002   17698                 1        4.605711  19.714524
4     10002   77632                 2        4.605711  19.714524


In [24]:
new_table = pd.merge(general_table, income_inspection_rating_by_zipcode, left_on='Zipcode', right_on='Zip_Code', how='left')
new_table = new_table.dropna()
print(new_table.shape)

(102, 14)


In [25]:
rest_rev = pd.read_csv("/content/ORIE4741Project/My Drive/ORIE4741Project/Data/New Data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv", encoding='latin-1')
rest_rev["ryear"] = pd.DatetimeIndex(rest_rev['INSPECTION DATE']).year
rest_rev_by_zipcode = rest_rev.groupby(['ZIPCODE', 'ryear']).mean()[['SCORE']]
rest_rev_by_zipcode = rest_rev_by_zipcode.reset_index()
zipcodes = rest_rev_by_zipcode.groupby('ZIPCODE')['ZIPCODE'].unique()
zipcodes_year_count = rest_rev_by_zipcode.groupby('ZIPCODE')['ZIPCODE'].count()
rest_change = pd.DataFrame(columns=['ZIPCODE', '2016_change', '2017_change','2018_change'])

for z in zipcodes:
  # check if zipcode valid
  if z[0] >= 10000:
    # if all four years data exist
    if zipcodes_year_count[z[0]] >= 4:
      # complaint_2015 = indoor_complaint_by_zipcode.at()
      try:
        rest_2015 = rest_rev_by_zipcode.loc[(rest_rev_by_zipcode['ZIPCODE'] == z[0]) & (rest_rev_by_zipcode['ryear'] == 2015)]['SCORE']
      except: 
        rest_2015 = 0
      try: 
        rest_2016 = rest_rev_by_zipcode.loc[(rest_rev_by_zipcode['ZIPCODE'] == z[0]) & (rest_rev_by_zipcode['ryear'] == 2016)]['SCORE']
        
      except:
        rest_2016 = 0
      try:
        rest_2017 = rest_rev_by_zipcode.loc[(rest_rev_by_zipcode['ZIPCODE'] == z[0]) & (rest_rev_by_zipcode['ryear'] == 2017)]['SCORE']
        
      except:
        rest_2017 =0
      try:
        rest_2018 = rest_rev_by_zipcode.loc[(rest_rev_by_zipcode['ZIPCODE'] == z[0]) & (rest_rev_by_zipcode['ryear'] == 2018)]['SCORE']
      except:
        rest_2018 = 0
      try:
        change_2016 = (rest_2016.iloc[0]-rest_2015.iloc[0]) / rest_2015.iloc[0]
        assert(not math.isinf(change_2016))
      except: 
        change_2016 = 0
      try: 
        change_2017 = (rest_2017.iloc[0]-rest_2016.iloc[0]) / rest_2016.iloc[0]
        assert(not math.isinf(change_2017))
      except:
        change_2017 = 0
      try:
        change_2018 = (rest_2018.iloc[0]-rest_2017.iloc[0]) / rest_2017.iloc[0]
        assert(not math.isinf(change_2018))
      except:
        change_2018 = 0

      
    # one of the four years data exist
    else:
      change_2016 = 0
      change_2017 = 0
      change_2018 = 0
    rest_change = rest_change.append({'ZIPCODE': z[0], '2016_change': change_2016, '2017_change': change_2017, '2018_change': change_2018}, ignore_index=True)
rest_change['ZIPCODE'] = rest_change['ZIPCODE'].astype(int)
new_table = pd.merge(new_table, rest_change, left_on='Zipcode', right_on='ZIPCODE', how='left')
new_table = new_table.dropna()
print(new_table.shape)
print(new_table)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in double_scalars


(102, 18)
     Zipcode  longtitude  latitude  ... 2016_change 2017_change 2018_change
0      10001     -73.990    40.750  ...    0.000000    0.036433    0.076492
1      10002     -73.990    40.720  ...    1.361153    0.120018    0.079009
2      10002     -73.990    40.720  ...    1.361153    0.120018    0.079009
3      10002     -73.990    40.720  ...    1.361153    0.120018    0.079009
4      10002     -73.990    40.720  ...    1.361153    0.120018    0.079009
..       ...         ...       ...  ...         ...         ...         ...
97     10128     -73.950    40.780  ...    0.000000    0.114572   -0.063232
98     10128     -73.950    40.780  ...    0.000000    0.114572   -0.063232
99     10128     -73.950    40.780  ...    0.000000    0.114572   -0.063232
100    10128     -73.950    40.780  ...    0.000000    0.114572   -0.063232
101    10282     -74.015    40.716  ...    0.000000    0.179316    0.064212

[102 rows x 18 columns]


In [0]:
# split the features and labels
features = np.array(new_table[['average_review', "longtitude", "latitude", "icc2016_change", "icc2017_change", "icc2018_change", '2016_change', '2017_change', '2018_change']])
labels = np.array(new_table["PriceChange"])
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.2, random_state = 42)

In [37]:
from sklearn.ensemble import RandomForestClassifier 
rfc = RandomForestClassifier(n_estimators = 300, max_depth=10, random_state = 10) 
# fit the regressor with x and y data 
rfc.fit(train_features, train_labels) 
print(rfc.feature_importances_)
# Use the forest's predict method on the test data
predictions = rfc.predict(test_features)
# Calculate the absolute errors
correct_predictions = (predictions == test_labels)
print("correct_predictions", correct_predictions)
print("accuracy:", np.sum(correct_predictions) / test_labels.size)
print("prediction:", predictions)
print("test_labels:", test_labels)

[0.11269174 0.11609556 0.11310713 0.09740819 0.12320665 0.14707087
 0.0455877  0.1103641  0.13446806]
correct_predictions [ True  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True]
accuracy: 0.9523809523809523
prediction: [ 4  8  6  2  2  6  8  2  1  6 10  2  3  6  1 10  1  7  1  4  3]
test_labels: [ 4  8  6  2  2  6  8  2  1 10 10  2  3  6  1 10  1  7  1  4  3]


## Linear Regression

In [39]:
from sklearn.linear_model import LinearRegression

# split the features and labels
features = np.array(new_table[['average_review', "longtitude", "latitude", "icc2016_change", "icc2017_change", "icc2018_change", '2016_change', '2017_change', '2018_change']])
labels = np.array(new_table['year over year'])

# get train/test split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

lr = LinearRegression()
lr_fit = lr.fit(train_features, train_labels)
train_score=lr.score(train_features, train_labels)
test_score=lr.score(test_features, test_labels)
print(train_score, test_score)

0.49924108446665916 0.38030852205802224


## Ridge Regression

In [40]:
from sklearn.linear_model import Ridge
rr = Ridge(alpha=0.01) 
# higher the alpha value, more restriction on the coefficients; low alpha > more generalization, coefficients are barely
# restricted and in this case linear and ridge regression resembles
rr.fit(train_features, train_labels)

# Use the forest's predict method on the test data
predictions = rr.predict(test_features)
# Calculate the absolute errors
correct_predictions = (abs(predictions - test_labels) < 0.05)
print("correct_predictions", correct_predictions)
print("accuracy:", np.sum(correct_predictions) / test_labels.size)
print("prediction:", predictions)
print("test_labels:", test_labels)

# Ridge_train_score = rr.score(train_features, train_labels)
# Ridge_test_score = rr.score(test_features, test_labels)

rr100 = Ridge(alpha=100) #  comparison with alpha value
rr100.fit(train_features, train_labels)

# Use the forest's predict method on the test data
predictions = rr100.predict(test_features)
# Calculate the absolute errors
correct_predictions = (abs(predictions - test_labels) < 0.05)
print("correct_predictions", correct_predictions)
print("accuracy:", np.sum(correct_predictions) / test_labels.size)
print("prediction:", predictions)
print("test_labels:", test_labels)

correct_predictions [ True  True  True  True  True False  True  True False False False False
  True  True  True False  True  True False  True  True  True  True  True
  True  True]
accuracy: 0.7307692307692307
prediction: [ 0.00988357  0.01885717 -0.01806609 -0.0328662  -0.0328662  -0.04859091
  0.05744736 -0.0328662   0.02575672  0.01967889  0.0184559   0.00567675
 -0.10153043  0.00803528 -0.10696951  0.05447624 -0.10696951  0.00129961
  0.02575672 -0.03113503 -0.10153043 -0.02088283 -0.03113503  0.01885717
 -0.0328662  -0.10153043]
test_labels: [-0.03184777  0.04180756  0.00050582 -0.07433545 -0.07433545  0.00202788
  0.05704622 -0.07433545 -0.0939048   0.12936047  0.11081162 -0.06017063
 -0.05185549  0.00453939 -0.14000858  0.11280746 -0.14000858  0.02420343
 -0.0939048  -0.03866348 -0.05185549  0.00828586 -0.03866348  0.04180756
 -0.07433545 -0.05185549]
correct_predictions [ True  True  True False False  True False False False False False False
  True  True False False False  True 

# Predict street score

To predict street score, we the features are the zipcode, longitude, latitude, indoor complaint change, cafe change, inspection change, and the housing price change. The labels are the street score for each zipcode.